In [1]:
import tensorflow as tf
import numpy as np

<h1>Construction Phase</h1>

In [2]:
n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [3]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [4]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1]) # no.of features
        
        stddev = 2/np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        
        if(activation is not None):
            return activation(Z)
        else:
            return Z

In [5]:
# Creating deep neural network

with tf.name_scope("dnn"):
    # Hidden layer 1
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    
    # Hidden layer 2
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    
    # Output layer
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

In [6]:
# Using tf.layers.dense to create deep neural networks

# with tf.name_scope("dnn"):
#     # Hidden layer 1
#     hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    
#     # Hidden layer 2
#     hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    
#     # Output layer
#     logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

In [7]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    
    loss = tf.reduce_mean(xentropy, name="loss")

In [8]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [9]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [10]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

<h1>Execution Phase</h1>

In [11]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [12]:
n_epochs = 50
batch_size = 50

In [15]:
with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        
        print "xentropy:\n", sess.run(xentropy, feed_dict={X: X_batch, y:y_batch})
        print "logits:\n",sess.run(logits, feed_dict={X: X_batch, y:y_batch})
        print "correct:\n",sess.run(correct, feed_dict={X: X_batch, y:y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
    
    save_path = saver.save(sess, "./my_model_final.ckpt")

xentropy:
[  7.19397189e-03   1.01011954e-01   5.65879792e-03   2.60810815e-02
   1.36564398e+00   1.22548722e-01   1.67485774e-01   3.68498087e-01
   3.99390556e-04   8.28247983e-03   7.97220394e-02   2.86109626e-01
   3.20385699e-03   1.18083131e+00   3.12464893e-01   8.70280340e-03
   1.73285261e-01   1.27139250e-02   1.56638682e-01   9.49545205e-03
   2.51783550e-01   2.81603307e-01   8.83229449e-02   4.12404947e-02
   1.61092475e-01   8.71381926e-05   7.38935843e-02   3.56692937e-03
   1.47986291e-02   1.08951382e-01   1.22762090e-02   1.14839873e-03
   5.43203413e-01   2.91078421e-03   6.76794350e-01   9.56602916e-02
   7.55174980e-02   3.88066232e-01   1.42955875e+00   5.98898623e-03
   1.02719285e-01   3.74487340e-02   8.72946111e-04   7.86759377e-01
   4.47173254e-04   2.09585037e-02   2.00582901e-03   9.30202827e-02
   2.07007094e-03   6.06990047e-02]
logits:
[[  5.11182249e-02  -4.32362032e+00   2.01404405e+00  -3.93587160e+00
    3.14643204e-01   1.19725227e+00   7.54915905

xentropy:
[  4.87890951e-02   1.45276129e-01   8.57090950e-03   1.40247405e+00
   9.77751240e-03   5.22764139e-02   1.84481218e-01   8.06811359e-03
   4.62597981e-02   2.97984132e-03   1.34919938e-02   1.81551203e-01
   3.15857083e-02   1.20920232e-02   2.27803905e-02   1.47973359e-01
   2.27406338e-01   4.78034170e-04   1.64963380e-02   5.46602048e-02
   2.30820075e-01   2.92564207e-03   7.86279023e-01   1.69811428e+00
   1.35967302e+00   2.78058678e-01   6.58069029e-02   5.63994572e-02
   2.44145896e-02   8.50117393e-03   2.49057233e-01   8.07010010e-02
   5.56337135e-03   5.98938391e-02   1.78724788e-02   3.18283746e-05
   7.97090586e-03   2.71050787e+00   3.74625254e+00   6.17790094e-04
   6.12439811e-02   1.56411331e-03   6.15160823e-01   1.86857414e-02
   2.03551427e-02   2.47987062e-01   5.06754398e-01   4.33184858e-03
   1.69393606e-02   4.70064059e-02]
logits:
[[  6.36677742e-01  -2.75957346e+00   2.23627940e-01  -1.08317959e+00
   -1.97991621e+00   5.10293102e+00   8.64101276

xentropy:
[  2.38574832e-03   1.99971814e-02   4.96536970e-01   3.95047888e-02
   3.25174350e-03   9.76747740e-03   8.42038453e-01   1.46083292e-02
   2.55327486e-02   9.53673862e-07   1.91992871e-03   6.20435812e-02
   1.16686024e-01   3.41302268e-02   8.90937671e-02   9.66972671e-03
   1.29841873e-03   6.97868243e-02   1.87258959e-01   2.78715929e-03
   3.51561303e-03   9.22854990e-03   1.01554347e-02   6.60599023e-02
   7.01457411e-02   2.32771109e-03   8.20880458e-02   6.44367607e-03
   9.99373719e-02   4.81966214e-04   2.99444515e-02   3.11473198e-03
   3.14434814e+00   2.54319306e-03   5.93644654e-05   6.59815133e-01
   1.72133034e-03   8.79403576e-02   5.33644482e-03   1.87720388e-01
   1.07992981e-02   5.17192818e-02   7.10958149e-03   7.15266494e-03
   1.27545805e-04   8.76885355e-01   3.33325239e-03   1.48450248e-02
   2.90027332e+00   1.71564233e+00]
logits:
[[ -8.14069748e-01   8.54964375e-01   3.23010755e+00   1.19682837e+00
   -4.21322632e+00   1.76514852e+00  -1.84202683

xentropy:
[  3.66789149e-03   4.80116189e-01   4.56705689e-04   9.87563431e-02
   2.21682172e-02   1.43702850e-02   5.71655083e+00   4.99384105e-02
   6.81982577e-01   2.49569081e-02   4.54950985e-03   5.81893399e-02
   2.21138890e-03   2.32110880e-02   2.66114697e-02   5.57198152e-02
   1.49834857e-04   1.46133184e+00   2.01458018e-02   3.19475803e-05
   1.08695542e-03   2.02273846e-01   1.45444181e-02   1.07118443e-01
   1.43955490e-02   1.75726891e-03   4.26506512e-02   6.38196524e-03
   2.01155432e-02   1.98582959e-04   3.24996118e-03   6.40683947e-03
   5.99017134e-03   2.67015435e-02   2.55674822e-03   1.45941144e-02
   1.55589223e-01   1.44541832e-02   1.01014599e-03   2.81175191e-04
   1.21471407e-02   8.54407996e-03   1.53916171e-02   1.78618226e-02
   1.51851093e-02   9.48098721e-04   6.92550698e-03   1.15884021e-02
   2.78608575e-02   1.98563691e-02]
logits:
[[ -1.76143920e+00   2.27291298e+00   1.06565199e+01   4.83629227e+00
   -6.09191751e+00  -4.08461601e-01   3.97436947

xentropy:
[  2.92744841e-02   1.04134150e-01   6.21031504e-03   5.63236373e-03
   1.32205427e-01   1.57721186e+00   1.16649747e-03   1.66318789e-02
   1.02503202e-03   1.61465615e-01   1.01514766e-03   1.50040425e-02
   2.92611751e-03   1.87980905e-01   1.07903279e-01   1.20681286e-01
   1.03415111e-02   1.85964764e-05   1.44908249e-01   4.76937788e-03
   1.72920916e-02   1.88923478e-02   1.82850330e-04   1.03515433e-03
   1.54979657e-02   5.68898141e-01   7.37787830e-03   3.63998901e-04
   5.99656999e-03   1.18364347e-03   1.43279307e-04   4.44068527e-03
   2.37749651e-01   3.40046274e-04   2.06329022e-03   9.79144219e-03
   1.32091984e-03   9.29728210e-01   2.99766451e-01   3.64232212e-01
   2.81005919e-01   1.89089705e-03   4.28503081e-02   2.03345846e-02
   1.82600701e+00   3.95816654e-01   1.65210426e-04   1.18304812e-03
   4.33192778e+00   3.21597746e-03]
logits:
[[  1.12808436e-01  -2.70489192e+00  -5.14412451e+00   1.77844024e+00
    1.87776172e+00   2.96836519e+00  -6.10619402

...

xentropy:
[  3.33785465e-06   1.06448228e-04   2.06229943e-05   2.13317592e-02
   5.06511191e-04   3.42124804e-05   1.65998712e-02   5.35775535e-02
   7.04993610e-04   2.79506727e-04   1.22420446e-04   3.81462451e-05
   2.93250559e-05   4.51537631e-02   6.89034583e-03   3.45089659e-02
   3.57627812e-07   6.63537125e-04   1.93934713e-04   1.77538507e-02
   4.09234613e-02   1.41468765e-02   8.38006163e-05   5.03628701e-03
   4.76836021e-06   3.21859916e-05   6.24637178e-05   9.95348018e-05
   2.67089042e-03   1.81114469e-02   1.23976897e-05   1.74387984e-04
   4.68481521e-05   7.58142342e-05   1.68503332e-03   4.10234206e-04
   4.43836878e-04   0.00000000e+00   4.04111343e-05   5.15264831e-02
   1.13957591e-04   5.10484576e-01   9.03565597e-05   3.33785465e-06
   5.73380239e-05   3.08747294e-05   4.62782569e-04   1.10898493e-03
   3.95339070e-04   3.99343153e-05]
logits:
[[  1.87935810e+01  -7.43699121e+00   5.92802143e+00   1.08036689e-01
   -7.78000212e+00  -1.39328587e+00   3.80950999

xentropy:
[  8.90453739e-05   1.52814639e-04   2.12226454e-02   4.89938175e-05
   5.25693991e-04   4.73387219e-04   1.32292643e-01   1.84264313e-02
   2.12408399e-04   2.50665704e-04   2.82522033e-05   1.97846559e-03
   2.05876790e-02   2.52720492e-05   1.38988369e-04   1.21495803e-03
   3.93389882e-06   1.68547733e-04   4.13647685e-05   1.18527906e-02
   0.00000000e+00   3.53750598e-04   9.77511354e-06   9.17907346e-06
   1.08318450e-02   5.02579263e-04   1.56271170e-04   6.07965512e-06
   2.01461662e-05   4.92689840e-04   4.88756905e-06   2.74180979e-06
   1.11029472e-03   2.02653737e-05   2.38418551e-07   2.51382263e-03
   9.53673862e-07   2.43819295e-03   8.22540733e-06   2.13382355e-05
   6.76641357e-04   1.20751720e-04   6.26367808e-04   6.75092684e-04
   2.62297299e-02   2.82019936e-02   3.50088277e-03   1.12177394e-02
   1.40163372e-03   1.26361047e-05]
logits:
[[ -9.16160393e+00   1.23824844e+01   7.18417585e-01   6.74035326e-02
    7.17239439e-01  -4.97833443e+00  -4.02201891

xentropy:
[  2.42404151e-03   2.44376060e-05   1.32746762e-03   1.32313551e-04
   2.22658156e-04   2.41214945e-03   3.04295012e-04   7.80790360e-05
   7.15255510e-07   7.59375617e-02   1.19209282e-07   3.57627812e-07
   5.00666465e-05   3.79078338e-05   2.67481385e-03   1.76428184e-05
   0.00000000e+00   1.19321383e-04   0.00000000e+00   2.62260096e-06
   1.22784813e-05   8.38901252e-02   2.18248388e-04   4.76837045e-07
   1.62826633e-04   4.77438414e-04   5.26122600e-02   8.46382409e-06
   2.26497400e-06   5.96046277e-07   1.78917137e-04   3.34691559e-03
   1.97656197e-03   6.37580594e-03   6.91389941e-05   2.71793488e-05
   6.38540043e-03   2.62260096e-06   0.00000000e+00   7.43716117e-03
   1.54971951e-06   1.51980290e-04   3.27820162e-05   1.52792968e-03
   2.59484950e-04   2.38418551e-07   1.24591426e-03   0.00000000e+00
   1.29068014e-03   1.27903390e-04]
logits:
[[ -1.66810548e+00  -7.27210331e+00  -3.45814180e+00   1.55300093e+00
    2.30427885e+00   1.08019698e+00  -6.08137369

xentropy:
[  4.76837045e-07   1.97126139e-02   6.45919237e-03   9.53673862e-07
   1.33282632e-01   4.58923906e-01   1.20155746e-04   0.00000000e+00
   1.57329254e-02   8.02246359e-05   1.90734681e-06   9.26213252e-05
   4.76836021e-06   5.86206745e-03   1.19209282e-07   1.83207900e-04
   2.16404721e-03   9.89432192e-06   9.46476939e-05   2.38418306e-06
   2.30376601e-01   2.78945845e-05   1.92861422e-03   3.94981558e-04
   3.61914337e-01   3.51661220e-05   1.43051045e-06   8.67348164e-04
   4.45028447e-04   1.09305382e-01   1.44955413e-02   3.14707577e-05
   7.79205875e-04   2.38418551e-07   6.38941347e-05   3.36651923e-03
   1.75103109e-04   3.12314838e-01   1.96375716e-02   4.55369118e-05
   4.76837045e-07   1.32671135e-04   2.84700207e-02   3.33437324e-02
   3.18238162e-04   6.90197994e-05   1.93338783e-04   1.58494210e-03
   2.10561943e+00   1.45902373e-02]
logits:
[[ -0.9785862   -1.85284448   1.88476539  -6.80281496  16.62370682
   -3.34164047   0.06450581  -0.68706918  -5.288565

xentropy:
[  1.42381504e-01   7.15253191e-06   2.49845795e-02   3.25507601e-04
   1.91957178e-03   6.82913605e-03   9.33211588e-04   1.23579404e-03
   5.78148356e-05   1.33513513e-05   3.97534715e-03   1.36962088e-04
   1.21745828e-03   5.38811473e-05   2.71760247e-04   3.60600883e-03
   8.63012206e-03   2.60676723e-04   1.09444577e-02   1.16303325e-01
   2.26497400e-06   4.04752820e-04   7.58645236e-02   2.46811919e-02
   1.01327387e-05   3.09943675e-06   1.97299267e-03   1.78202012e-04
   2.38418551e-07   9.33687959e-04   2.50339190e-06   1.39154242e-02
   2.38418551e-07   2.77361542e-04   1.07702389e-01   9.53669769e-06
   5.96046277e-07   4.52994254e-06   2.05257325e-04   2.18640990e-03
   3.61973100e-04   3.36164558e-05   3.13515520e-05   6.61102310e-03
   2.31263348e-05   4.70289262e-04   1.01322759e-04   7.31953839e-03
   6.81644771e-04   0.00000000e+00]
logits:
[[  2.55127120e+00   2.48220658e+00   3.41573143e+00  -3.38590193e+00
    6.45539379e+00  -1.41821861e+00   3.43748903

In [17]:
mnist.train.num_examples

55000

In [19]:
mnist.train.images[0, :]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [20]:
mnist.train.num_examples // batch_size

1100